# Getting Started with LangChain
This notebook shows how to assemble a basic LangChain chain of tasks using the OpenAI API as the backend. This tutorial will require you to have your own OpenAI API key.

## Notebook Setup

In [4]:
# Importing the necessary Python libraries
import os
import yaml
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper

In [5]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/openai-keys.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Setting the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = keys_yaml['API_KEY']

In [15]:
# Using LangChain to connect to OpenAI
openai_llm = OpenAI()

In [16]:
# Instantiating an object to obtain results from the Wikipedia API
wikipedia_api = WikipediaAPIWrapper(top_k_results = 1)

## Prompt Templates
In order to create a chained set of tasks, we need to instantiate the individual tasks with **prompt templates**. Prompt templates combine an input alongside some preset feature engineering to produce a precise result (also known as a Completion).

In [7]:
# Creating a prompt template for checking to see if the inputted individual is a historical figure
is_historical_figure_template = PromptTemplate(
    input_variables = ['entity_name'],
    template = 'Is the following entity a person, and if yes, would you consider this person to be a historical figure: {entity_name}. Please give me back just a yes or no answer.'
)

# Creating a prompt template for generating a research paper outline the three most important events in the historical figure's life
research_paper_outline_template = PromptTemplate(
    input_variables = ['entity_name', 'wikipedia_entry'],
    template = 'The following is a Wikipedia entry about {entity_name}. Please provide for me an outline of a basic research paper with an introduction, the three most important events of this person\'s life, and a conclusion. Here is the Wikipedia information:\n\n {wikipedia_entry}'
)

# Creating a prompt template for generating a research paper based on the outline generated from the previous prompt template
research_paper_template = PromptTemplate(
    input_variables = ['entity_name', 'research_paper_outline'],
    template = 'You are a high schooler who has just been assigned the task of writing a short research paper about a historical figure. The historical figure is {entity_name}, and the following is an outline to follow:\n {research_paper_outline}'
)

In [10]:
# Instantiating objects to hold the intermediate memory of each LangChain step
research_paper_outline_memory = ConversationBufferMemory(input_key = 'research_paper_outline_prompt', memory_key = 'chat_history')
research_paper_memory = ConversationBufferMemory(input_key = 'research_paper_prompt', memory_key = 'chat_history')

In [11]:
# Instantiating the LangChains objects for all the prompt templates defined above
is_historical_figure_chain = LLMChain(llm = openai_llm, prompt = is_historical_figure_template, verbose = True, output_key = 'is_historical_figure')
research_paper_outline_chain = LLMChain(llm = openai_llm, prompt = research_paper_outline_template, verbose = True, output_key = 'research_paper_outline', memory = research_paper_outline_memory)
research_paper_chain = LLMChain(llm = openai_llm, prompt = research_paper_template, verbose = True, output_key = 'research_paper', memory = research_paper_memory)

## Instantiating the LangChain

In [13]:
# Creating the LangChain chain
research_paper_langchain = SequentialChain(chains = [research_paper_outline_chain, research_paper_chain],
                                           input_variables = ['entity_name', 'wikipedia_entry'],
                                           output_variables = ['research_paper'],
                                           verbose = True)

## Using the LangChain

In [ ]:
# Testing the 